In [4]:
import numpy as np
import pandas as pd
import scipy as sp

from copulas.multivariate import GaussianMultivariate

In [5]:
usaid = pd.read_csv("USAID_GHSC-PSM_Health_Commodity_Delivery_Dataset_20240105.csv", 
                    low_memory= False, header= 0)
features = ["Average Days Late", "Illustrative Price", "Shipped Quantity", "Order Cycle Time"]
usaid = usaid[features]
usaid.dropna(inplace= True)
usaid.to_csv("usaid.csv")

In [3]:
num_distributors = 15
chunk_size = usaid.shape[0] // num_distributors
chunks = [usaid[i:i+chunk_size].copy().reset_index(drop= True) for i in range(0, usaid.shape[0], chunk_size)]    
dataset = pd.concat(chunks, axis= 1)
# dataset.to_csv("usaid.csv")
# dataset = dataset.to_numpy()
# dataset = usaid.to_numpy()

In [ ]:
synthesizer = GaussianMultivariate()
synthesizer.fit(dataset)


In [ ]:
num_scenarios = 2
scenarios = []

for i in range(num_scenarios):
    copula = synthesizer.sample(dataset.shape[0])
    scenarios.append(copula)
scenarios = np.array(scenarios)
print(scenarios.shape)

In [ ]:
from metrics import CPRS

# for scenario in scenarios:
#     assert(scenario.shape == dataset.shape)

#     all_cprs_scores = []; individual_cprs_score = []
#     for idx in range(dataset.shape[1]):
#         score = CPRS(scenario[:, idx], dataset[:, idx])
#         individual_cprs_score.append(score)
#     mean_individual_cprs = np.mean(np.array(individual_cprs_score), axis= -1)
#     all_cprs_scores.append(mean_individual_cprs)
# print(np.mean(np.array(all_cprs_scores), axis= -1))

scenario = scenarios[0]
individual_cprs_score = []
for idx in range(dataset.shape[1]):
    score = CPRS(scenario[:, idx], dataset[:, idx])
    individual_cprs_score.append(score)
    print(score)
mean_individual_cprs = np.mean(np.array(individual_cprs_score), axis= -1)




In [ ]:
np.savetxt("dataset/USAID_copula.txt", scenarios.reshape(-1))
np.savetxt("dataset/USAID_preprocessed.txt", dataset.reshape(-1))